# Vector Store

In [12]:
from devtools import debug
from dotenv import load_dotenv

load_dotenv(verbose=True)

%load_ext autoreload
%autoreload 2

!export PYTHONPATH=":./python"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Split the text into chunks

In [13]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from python.ai_core.embeddings import EmbeddingsFactory
from python.ai_core.vector_store import VECTOR_STORE_ENGINE, VectorStoreFactory

loader = TextLoader("use_case_data/other/state_of_the_union.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
debug(texts)

/tmp/ipykernel_1371025/2073757387.py:11 <module>
    texts: [
        Document(
            metadata={
                'source': 'use_case_data/other/state_of_the_union.txt',
            },
            page_content=(
                'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the'
                ' Cabinet. Justices of the Supreme Court. My fellow Americans.  \n'
                '\n'
                'Last year COVID-19 kept us apart. This year we are finally together again. \n'
                '\n'
                'Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n'
                '\n'
                'With a duty to one another to the American people to the Constitution. \n'
                '\n'
                'And with an unwavering resolve that freedom will always triumph over tyranny. \n'
                '\n'
                'Six days ago, Russia’s Vladimir Putin sought to shake 

[Document(metadata={'source': 'use_case_data/other/state_of_the_union.txt'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their cou

### Calculate embeddings and oyt them in a vector database .

This is a adaptation of https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/ <br><br>
We have a factory to facilitate Vector Store configuration and selection. <br>
One benefit if that it's easy to change vector store type and embedding model, just by configuration.

In [16]:
vs_engine: VECTOR_STORE_ENGINE | None = None
vs_engine = "InMemory"
vs_engine = "Chroma_in_memory"
vs_engine = "Sklearn"

vs_factory = VectorStoreFactory(
    id=vs_engine,
    collection_name="maintenance_procedure",
    embeddings_factory=EmbeddingsFactory(),
)

debug(vs_factory)

db = vs_factory.vector_store
db.add_documents(texts)

2024-11-05 14:53:15.228 | INFO     | python.ai_core.vector_store:vector_store:148 - get vector store  : Sklearn/maintenance_procedure_multilingual_MiniLM_local


/tmp/ipykernel_1371025/1436956741.py:12 <module>
    vs_factory: VectorStoreFactory(
        id='Sklearn',
        embeddings_factory=EmbeddingsFactory(
            embeddings_id='multilingual_MiniLM_local',
            encoding_str=None,
            retrieving_str=None,
            info=EmbeddingsInfo(
                id='multilingual_MiniLM_local',
                cls='HuggingFaceEmbeddings',
                model='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
                key=None,
                prefix='',
            ),
        ),
        collection_name='maintenance_procedure',
        index_document=False,
        collection_metadata=None,
        collection_full_name='maintenance_procedure_multilingual_MiniLM_local',
        description='Sklearn/maintenance_procedure_multilingual_MiniLM_local',
        vector_store=<langchain_community.vectorstores.sklearn.SKLearnVectorStore object at 0x7f382179b8e0>,
    ) (VectorStoreFactory)


['b36bdbc2-f6b9-4356-bf3b-f263f7950d37',
 '23b4fe34-d945-443e-acec-d407e17bc240',
 'fea99fc4-14be-461f-86fa-0a64f6005768',
 '709236f7-4fc5-43ac-97ac-8cf1006f39fa',
 'f98b538b-12f3-4092-aee7-07e9cd05546b',
 'b9fdb96d-065f-4487-8fc2-33f5fad0dc1d',
 'bd114eef-966d-4915-9b61-87470b593ed7',
 'c1a59ff8-7f24-445b-b187-985d33a6d25d',
 'aee9a4ce-723a-40a0-b789-72b970ddfdfb',
 '9d8356a7-649e-4332-80dc-c88252fea57e',
 '2fb44f2c-1da1-4c5f-af59-c772299df6a3',
 '41fe56f7-97c7-4457-a0e9-a2e2bc8e130c',
 '517bab5d-0f31-4e29-9872-0f2051c3665d',
 '40eea1aa-f671-4302-8e2f-a0e1a7eff513',
 'b8b9b77e-dc24-4e74-9ea1-8211594f28c0',
 'eb7cb3a4-9635-45dc-8e8f-bf73e7fc0ab6',
 '166012eb-61b0-471e-89d0-a8fb39ccaeab',
 '9dc9ef1e-c879-4b10-a768-c9765dfd5223',
 'a89f3c75-581b-4655-abe0-7c73a50bae60',
 'f3d2207d-2be2-4e5c-aeca-74fd3c6bfadb',
 '381cab4a-25e5-440d-9dea-3dbce2753601',
 'bba18974-ebff-4f84-a0ee-44e64dae12c5',
 '3fbe882c-fe0b-4a12-841d-e8cc379b4100',
 '109f501d-145c-41ac-a557-d3e67cac75d5',
 '44355d7a-9734-

### Test a query

In [17]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query, k=3)
debug(docs)

/tmp/ipykernel_1371025/3766655050.py:3 <module>
    docs: [
        Document(
            metadata={
                'id': 'a3ede769-6e2e-4ca0-89db-d8ffdccac3cb',
                'source': 'use_case_data/other/state_of_the_union.txt',
            },
            page_content=(
                'Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act'
                '. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n'
                '\n'
                'Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen '
                'Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Cou'
                'rt. Justice Breyer, thank you for your service. \n'
                '\n'
                'One of the most serious constitutional responsibilities a President has is nominating someone to serv'
           

[Document(metadata={'id': 'a3ede769-6e2e-4ca0-89db-d8ffdccac3cb', 'source': 'use_case_data/other/state_of_the_union.txt'}, page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'),
 Document(metadata={'id': 'fea99fc4-14be-461f-86fa-0a64f6005768', 

### Vector Store as Runnable

In [18]:
retriever = db.as_retriever()

a = retriever.invoke(query)
debug(a)

/tmp/ipykernel_1371025/4059562495.py:4 <module>
    a: [
        Document(
            metadata={
                'id': 'a3ede769-6e2e-4ca0-89db-d8ffdccac3cb',
                'source': 'use_case_data/other/state_of_the_union.txt',
            },
            page_content=(
                'Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act'
                '. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n'
                '\n'
                'Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen '
                'Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Cou'
                'rt. Justice Breyer, thank you for your service. \n'
                '\n'
                'One of the most serious constitutional responsibilities a President has is nominating someone to serv'
              

[Document(metadata={'id': 'a3ede769-6e2e-4ca0-89db-d8ffdccac3cb', 'source': 'use_case_data/other/state_of_the_union.txt'}, page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'),
 Document(metadata={'id': 'fea99fc4-14be-461f-86fa-0a64f6005768', 

In [ ]:
type(retriever)

### Configurable Retriever Runnable 

It's a little bit tricky, so there is a small wrapper in the Factory: 

In [19]:
retriever = vs_factory.get_configurable_retriever(default_k=20)

config = {"configurable": {"search_kwargs": {"k": 1}}}
result = retriever.invoke(query, config=config)
result  # return 1 result

[Document(metadata={'id': 'a3ede769-6e2e-4ca0-89db-d8ffdccac3cb', 'source': 'use_case_data/other/state_of_the_union.txt'}, page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.')]